### Error Mitigation Workflow with mitiq on Amazon Braket

In this notebook, we will show how we can use the previous three methodologies, namely readout error mitigation, Pauli twirling, and Zero-noise extrapolation, in conjunction with ProgramSets in an applied molecular example. 

To coordinate our error mitigation pipeline, we will utilize some prebuilt functions for distributing circuits to ProgramSets in the `tools` folder.

In [4]:
def ising_hamiltonian(
        hopping : float, 
        self_interaction : float, 
        num_qubits : int):
    hamiltonian = []
    n = num_qubits
    for i in range(num_qubits):
        hamiltonian.append(
            (self_interaction,i*"I"+"Z"+(n-i-1)*"I")
            )
        if i>0:
            hamiltonian.append(
                (hopping, (i-1)*"I"+"XX"+(n-i-1)*"I"))
    return hamiltonian

ising_hamiltonian(0.5, 0.3, 10)


[(0.3, 'ZIIIIIIIII'),
 (0.3, 'IZIIIIIIII'),
 (0.5, 'XXIIIIIIII'),
 (0.3, 'IIZIIIIIII'),
 (0.5, 'IXXIIIIIII'),
 (0.3, 'IIIZIIIIII'),
 (0.5, 'IIXXIIIIII'),
 (0.3, 'IIIIZIIIII'),
 (0.5, 'IIIXXIIIII'),
 (0.3, 'IIIIIZIIII'),
 (0.5, 'IIIIXXIIII'),
 (0.3, 'IIIIIIZIII'),
 (0.5, 'IIIIIXXIII'),
 (0.3, 'IIIIIIIZII'),
 (0.5, 'IIIIIIXXII'),
 (0.3, 'IIIIIIIIZI'),
 (0.5, 'IIIIIIIXXI'),
 (0.3, 'IIIIIIIIIZ'),
 (0.5, 'IIIIIIIIXX')]

In [6]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir))) # parent directory 
from tools.observable_tools import pauli_grouping


bases, pauli_terms = pauli_grouping(ising_hamiltonian(0.5,1,10))
print(bases)
print(f'number of distinct circuits: {len(bases)}')


['ZZZZZZZZZZ', 'XXXXXXXXXX']
number of distinct circuits: 2


Now let's construct our pipeline. We will use essentially two steps, in the first, we characterize our readout error. In the second, we will combine observable measurement, twirling, and zero-noise extrapolation together. 



In [ ]:
from tools.program_set_tools import run_with_program_sets



In [16]:
from braket.circuits import Circuit
import numpy as np
from braket.parametric import FreeParameter

def test_circuit(
        theta1 : FreeParameter,
        theta2 : FreeParameter,
        num_qubits : int):
    circ = Circuit()
    for i in range(num_qubits):
        circ.prx(i,theta1,0)
    for i in range(0,num_qubits,2):
        circ.cz(i,i+1)
    for i in range(num_qubits):
        circ.prx(i,theta2, 0)        
    for i in range(1,num_qubits,2):
        circ.cz(i,i+1)
    return circ

alp = FreeParameter("alp")
bet = FreeParameter("bet")

ansatz = test_circuit(alp, bet, 10)

In [ ]:
from noise_models import qd_depol


def readout_step()
    # first, generate permutations

    # we use a recursive approach here -> 

    # 

